In [1]:
from elftools.elf.elffile import ELFFile

f = open("vmlinux-5.4.0-26-generic", "rb")
elffile = ELFFile(f)

In [7]:
from depsurf import get_section_info

get_section_info(elffile)

,name,sh_type,sh_flags,sh_addr,sh_offset,sh_size,sh_link,sh_info,sh_addralign,sh_entsize,data
0,,SHT_NULL,0,0,0,0,0,0,0,0,
1,.head.text,SHT_PROGBITS,6,18446603336490156032,65536,4096,0,0,4096,0,4d5a0091ffbf4e1400000800000000
2,.text,SHT_PROGBITS,6,18446603336490160128,69632,13854216,0,0,2048,8,fd7bbea9fd030091f30b00f9f30300
3,.rodata,SHT_PROGBITS,3,18446603336504049664,13959168,6235530,0,0,4096,0,696e697463616c6c5f626c61636b6c
4,.modinfo,SHT_PROGBITS,2,18446603336510285194,20194698,70832,0,0,1,0,6b766d2e7061726d747970653d6861
5,.init.eh_frame,SHT_PROGBITS,2,18446603336510356032,20265536,7264,0,0,8,0,1000000000000000017a520004781e
6,.pci_fixup,SHT_PROGBITS,2,18446603336510363296,20272800,9280,0,0,16,0,8680ffff000200000800000054be32
7,__ksymtab,SHT_PROGBITS,2,18446603336510372576,20282080,63876,0,0,4,0,507701ff64ba020000000000044828
8,__ksymtab_gpl,SHT_PROGBITS,2,18446603336510436452,20345956,70836,0,0,4,0,9898d1fe4b1e0100000000001c99d1
9,__ksymtab_strings,SHT_PROGBITS,2,18446603336510507288,20416792,222317,0,0,1,0,6b696d6167655f7661646472006c6f


In [14]:
dwarfinfo = elffile.get_dwarf_info()

In [16]:
from collections import defaultdict

from depsurf import reload_depsurf

reload_depsurf()

from depsurf import (
    DIEHandler,
    Traverser,
    disable_dwarf_cache,
    get_name,
    SubroutineEntry,
)
from elftools.dwarf.die import DIE


subroutine_info = {}


def filter_subprogram(die: DIE):
    assert die.tag == "DW_TAG_subprogram"

    # ignore declaration-only subprograms
    decl = die.attributes.get("DW_AT_declaration")
    if decl is not None and decl.value:
        return False

    # ignore inlined subprograms as they will be accounted at the call site
    aa = die.attributes.get("DW_AT_abstract_origin")
    if aa is not None:
        return False

    return True


def get_entry(die: DIE, traverser: Traverser):
    name = get_name(die)
    if name is None:
        return None, None

    info_dict = subroutine_info.get(name)
    if info_dict is None:
        info_dict = defaultdict(SubroutineEntry)
        subroutine_info[name] = info_dict

    extern = die.attributes.get("DW_AT_external")
    if extern is not None:
        assert extern.value
        info = info_dict[name]
        info.external = True
    else:
        location = traverser.get_die_location(die)
        info = info_dict[location]
        info.external = False
    return info, name


def handle_subprogram(die: DIE, traverser: Traverser):
    get_entry(die, traverser)


def resolve_abstract_origin(die: DIE):
    if "DW_AT_abstract_origin" not in die.attributes:
        return die

    return resolve_abstract_origin(die.get_DIE_from_attribute("DW_AT_abstract_origin"))


def handle_call_impl(die: DIE, traverser: Traverser, is_inline: bool):
    fn_die = resolve_abstract_origin(die)
    entry, name = get_entry(fn_die, traverser)
    if entry is None:
        return

    if name == "vfs_read" and is_inline:
        print("found vfs_read")

    caller = traverser.curr_subprogram
    assert caller is not None
    if is_inline:
        entry.caller_inline.append(caller)
    else:
        entry.caller_func.append(caller)


def handle_call_site(die: DIE, traverser: Traverser):
    handle_call_impl(die, traverser, is_inline=False)


def handle_inlined_subroutine(die: DIE, traverser: Traverser):
    handle_call_impl(die, traverser, is_inline=True)


handler_map = {
    "DW_TAG_compile_unit": DIEHandler(show=True, rec=True),
    "DW_TAG_subprogram": DIEHandler(
        show=True,
        rec=True,
        filter_fn=filter_subprogram,
        handle_fn=handle_subprogram,
    ),
    "DW_TAG_inlined_subroutine": DIEHandler(
        show=True,
        rec=True,
        handle_fn=handle_inlined_subroutine,
    ),
    "DW_TAG_GNU_call_site": DIEHandler(
        show=True,
        rec=False,
        handle_fn=handle_call_site,
    ),
    "DW_TAG_lexical_block": DIEHandler(show=False, rec=True),
}

smoke = True

if smoke:
    Traverser.enable_show = True
    cu_iter = list(dwarfinfo.iter_CUs())[:2]

    # disable_dwarf_cache()
    # cu_iter = [cu for cu in dwarfinfo.iter_CUs() if "fs/" in cu.get_top_DIE().get_full_path()]
else:
    disable_dwarf_cache()
    cu_iter = dwarfinfo.iter_CUs()


for cu in cu_iter:
    traverser = Traverser(cu, handler_map)
    traverser.traverse()

compile_unit: /build/linux-xMsRmn/linux-5.4.0/arch/arm64/kernel/head.S
compile_unit: /build/linux-xMsRmn/linux-5.4.0/init/main.c
  subprogram: kernel_init
    inlined_subroutine: mark_readonly
      inlined_subroutine: rodata_test
      call_site: rcu_barrier
      call_site: mark_rodata_ro
      call_site: printk
    call_site: kernel_init_freeable
    call_site: async_synchronize_full
    call_site: ftrace_free_init_mem
    call_site: free_initmem
    call_site: numa_default_policy
    call_site: rcu_end_inkernel_boot
    call_site: run_init_process
    call_site: printk
    call_site: run_init_process
    call_site: panic
    call_site: try_to_run_init_process
    call_site: try_to_run_init_process
    call_site: try_to_run_init_process
    call_site: try_to_run_init_process
    call_site: panic
  subprogram: free_initmem
    inlined_subroutine: free_initmem_default
      call_site: free_reserved_area
  subprogram: mark_readonly (Not declared inline but inlined by the compiler)
  su

In [5]:
import pickle

with open("subroutine_info.pkl", "wb") as f:
    pickle.dump(subroutine_info, f)

# Analysis

In [2]:
import pickle

with open("subroutine_info.pkl", "rb") as f:
    subroutine_info = pickle.load(f)

In [23]:
from collections import defaultdict

sizes = defaultdict(int)

In [24]:
for name, v in sorted([(name, v) for name, v in subroutine_info.items() if len(v) > 1], key=lambda x: -len(x[1])):
    sizes[len(v)] += 1
    print(name)
    for location, info in v.items():
        print(f"\t{location:40}")

type_show
	kernel/irq/irqdesc.c                    
	kernel/events/core.c                    
	drivers/video/backlight/backlight.c     
	drivers/xen/sys-hypervisor.c            
	drivers/regulator/core.c                
	drivers/base/cacheinfo.c                
	drivers/usb/core/endpoint.c             
	drivers/input/serio/serio.c             
	drivers/thermal/thermal_sysfs.c         
	drivers/mmc/core/bus.c                  
	drivers/firmware/memmap.c               
	drivers/nvmem/nvmem-sysfs.c             
	net/core/net-sysfs.c                    
	net/rfkill/core.c                       
name_show
	kernel/irq/irqdesc.c                    
	drivers/regulator/core.c                
	drivers/base/power/wakeup_stats.c       
	drivers/rtc/sysfs.c                     
	drivers/i2c/i2c-dev.c                   
	drivers/pps/sysfs.c                     
	drivers/hwmon/hwmon.c                   
	drivers/remoteproc/remoteproc_sysfs.c   
	drivers/devfreq/devfreq.c               
	drivers/devfr

In [27]:
import pandas as pd

df = pd.DataFrame.from_dict(sizes, orient="index")

# sizes

df

,0
14,1
13,2
12,1
9,1
7,1
6,3
5,12
4,20
3,41
2,423


: 

In [4]:
flattened_info = [entry for v in subroutine_info.values() for entry in v.values()]

len(flattened_info)

79338

In [14]:
len([len(d) for d in subroutine_info.values() if len(d) > 1])

505

In [5]:
sum([len(d) for d in subroutine_info.values() if len(d) > 1])

1195

In [6]:
sum([len(d) for d in subroutine_info.values() if len(d) > 1]) / len(
    [v for v in flattened_info if not v.external]
)

0.02088613125928515

In [7]:
len([v for v in flattened_info if v.external]) / len(flattened_info)

0.2788449418941743

In [8]:
import pandas as pd

pd.set_option("display.max_rows", None)

df = pd.DataFrame(
    [
        {
            "name": name,
            "location": location,
            "caller_inline": len(info.caller_inline),
            "caller_func": len(info.caller_func),
            "external": info.external,
        }
        for name, d in subroutine_info.items()
        for location, info in d.items()
    ]
)

print(len(df))

df[:10]

79338


,name,location,caller_inline,caller_func,external
0,kernel_init,init/main.c,0,0,False
1,mark_readonly,init/main.c,1,0,False
2,rodata_test,include/linux/rodata_test.h,1,0,False
3,rcu_barrier,rcu_barrier,1,35,True
4,mark_rodata_ro,mark_rodata_ro,0,1,True
5,printk,printk,0,5936,True
6,kernel_init_freeable,init/main.c,0,1,False
7,async_synchronize_full,async_synchronize_full,0,17,True
8,ftrace_free_init_mem,ftrace_free_init_mem,0,1,True
9,free_initmem,free_initmem,0,1,True


In [11]:
full_inlined_df = df[(df.caller_inline > 0) & (df.caller_func == 0)]
print(len(full_inlined_df) / len(df))

0.34888704025813605


In [12]:
df.caller_inline.sum() / (df.caller_inline.sum() + df.caller_func.sum())

0.6967692554561682

In [13]:
partial_inlined_df = df[(df.caller_inline > 0) & (df.caller_func > 0)]


print(len(partial_inlined_df))


partial_inlined_df[:10]

4118


,name,location,caller_inline,caller_func,external
3,rcu_barrier,rcu_barrier,1,35,True
19,kstrtobool,kstrtobool,1,47,True
25,do_raw_spin_lock,include/linux/spinlock.h,3342,204,False
30,__cmpxchg_case_acq_32,arch/arm64/include/asm/cmpxchg.h,5542,10,False
31,system_uses_lse_atomics,arch/arm64/include/asm/lse.h,12096,91,False
36,arch_local_irq_save,arch/arm64/include/asm/irqflags.h,4643,110,False
37,arch_local_save_flags,arch/arm64/include/asm/irqflags.h,4779,26,False
38,arch_irqs_disabled_flags,arch/arm64/include/asm/irqflags.h,4771,14,False
39,arch_local_irq_disable,arch/arm64/include/asm/irqflags.h,5386,140,False
45,arch_local_irq_restore,arch/arm64/include/asm/irqflags.h,4824,254,False
